In [32]:
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
plt.style.use('seaborn-darkgrid')

## What is probabilistic programming?

> Programs and languages as formal representations of probabilistic objects.

> probabilistic programs are usual functional or imperative programs with two added constructs: the ability
to draw values at random from distributions, and the ability to condition values of variables in a program via observations.

> Probabilistic Programming allows for automatic Bayesian inference on user-defined probabilistic models.

## What is probabilistic programming?

$\mu \sim \mathcal{N}(0, 1)$

$\sigma \sim \mathcal{U}(0, 1)$

$x_i \sim \mathcal{N}(\mu, \sigma)$

$\text{We observe } X = \{x_1, x_2, \ldots\}$

$\text{What is }P(\mu | X)\text{?}$

---

Compare it with "standard" machine learning, where we want to find


$\text{What is }argmax_\mu P(\mu | X)\text{?}$

## Probabilistic Programming Languages

* OpenBUGS -> WinBUGS -> JAGS, Stan, PyMC3, Pyro
* BayesDB
* ProbLog
* Anglican, Venture, Church, WebPPL, Hakaru (?)

# The math

$$p(\Theta | D) = \frac{p(D|\Theta)p(\Theta)}{p(D)}$$

* REALLY hard to compute $p(D)$ in general!
  * Marginalization, $p(D) = \int_{\Theta} p(D|\Theta)p(\Theta) d\Theta$ is rarely analytically tractable, and numerically hard in multidimensional cases.
* Easy to compute $f(\Theta) = p(D|\Theta)p(\Theta)$, which allows gradient descent family of algorithms. Hence the popularity of MLE methods.

But what if we really want the posterior?

# Metropolis-Hastings

$$f(\theta_1, \theta_2) = \frac{p(\theta_1 | D)}{p(\theta_2 | D)} = \frac{p(D|\theta_1)p(\theta_1)}{p(D|\theta_2)p(\theta_2)}$$
$$\theta_1 \succ \theta_2 \iff f(\theta_1, \theta_2) > 1$$

---

1. Start with some $\theta$.
2. Choose some $\theta' = \theta \pm \varepsilon$.
3. If $\theta' \succ \theta$, set $\theta = \theta'$.
4. If $\theta' \not\succ \theta$, set $\theta = \theta'$ with probability $f(\theta', \theta)$.
5. Rinse and repeat.

In [33]:
def sample_circle(n):
    RNG = np.random.RandomState(42)
    points = RNG.uniform(size=(n,2))
    flag = (points[:,0]**2 + points[:,1]**2) <= 1
    ratio = flag.mean() * 4
    plt.figure(figsize=(4,4))
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.scatter(points[flag,0],points[flag,1], marker='+')
    plt.scatter(points[np.logical_not(flag),0],
                points[np.logical_not(flag),1], marker='+')
    plt.show()
    print(ratio)

interact(sample_circle, n=(1, 20001, 500));

interactive(children=(IntSlider(value=10001, description='n', max=20001, min=1, step=500), Output()), _dom_cla…

In [34]:
def sample_circle(n):
    RNG = np.random.RandomState(42)
    def f(x, y):
        if x ** 2 + y ** 2 < 1:
            return 1
        else:
            return 0
    points = np.zeros(shape=(n,2))
    x0 = (0.9, 0.9)
    for i in range(n):
        points[i] = x0
        while True:
            (x, y) = x0
            x = (x + RNG.normal() * 0.2) % 1
            y = (y + RNG.normal() * 0.2) % 1
            # Jump only if the next point is "good".
            if f(x, y) > 0:
                x0 = (x, y)
                break

    plt.figure(figsize=(4,4))
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.plot(points[:,0],points[:,1], marker='+', alpha=0.8)
    plt.show()

In [35]:
interact(sample_circle, n=(0, 1000, 50));

interactive(children=(IntSlider(value=500, description='n', max=1000, step=50), Output()), _dom_classes=('widg…

In [36]:
def sample_normal(n):
    RNG = np.random.RandomState(42)
    # Non-normalized N(0, 1)
    def f(x): return np.exp(-x ** 2 / 2)
    
    points = np.zeros(shape=n)
    x0 = 2.0   # Starting point
    p0 = f(x0) # "Probability"
    for i in range(n):
        points[i] = x0
        while True:
            x1 = x0 + (2 * RNG.uniform() - 1) * 1
            p1 = f(x1)
            r = p1 / p0 # Probability ratio
            if r > 1 or RNG.uniform() < r:
                x0 = x1
                p0 = p1
                break

    plt.figure(figsize=(4,4))
    lx = np.linspace(-3, 3, 1000)
    ly = 1.0/np.sqrt(2 * np.pi * 1) * f(lx)
    plt.plot(lx, ly)
    plt.hist(points, bins=100, density=True)
    plt.show()

In [37]:
interact(sample_normal, n=(0, 20000, 500));

interactive(children=(IntSlider(value=10000, description='n', max=20000, step=500), Output()), _dom_classes=('…

In [55]:
def sample_normal(n, obs):
    RNG = np.random.RandomState(42)
    
    observed = RNG.normal(size=obs) * 1 + 2
    print(observed)
    
    def f(mu): return np.exp(-mu ** 2 / (2 * 4**2)) * np.product(np.exp(-(mu - observed) ** 2 / 2))
    
    points = np.zeros(shape=n)
    x0 = 2.0
    p0 = f(x0)
    for i in range(n):
        points[i] = x0
        while True:
            x1 = x0 + (2 * RNG.uniform() - 1) * 1
            p1 = f(x1)
            r = p1 / p0
            if r > 1 or RNG.uniform() < r:
                x0 = x1
                p0 = p1
                break

    plt.figure(figsize=(4,4))
    lx = np.linspace(-3, 3, 1000)
    ly = np.array([f(x) * n for x in lx])
    # plt.plot(lx, ly)
    plt.hist(points, bins=100)
    plt.show()

In [58]:
interact(sample_normal, n=(0, 200000, 50000),obs=(0,30,1));

interactive(children=(IntSlider(value=100000, description='n', max=200000, step=50000), IntSlider(value=15, de…

$\mu \sim N(0, 1)$

$\sigma \sim |N(0, 1)|$

$x \sim N(\mu, \sigma)$

$\{x_1, x_2, \ldots\}$

In [40]:
# Probabilistic Language Primitives
Sample  = namedtuple('Sample', 'distribution')
Observe = namedtuple('Observe', 'distribution value')

# Distributions
Uniform    = namedtuple('Uniform', 'min max')
Normal     = namedtuple('Normal', 'mu sigma')
HalfNormal = namedtuple('HalfNormal', 'sigma')
        
# Sample program.
def program(data):
    mu    = yield Sample(Normal(0, 1))
    sigma = yield Sample(HalfNormal(1))
    x = Normal(mu, sigma)
    for d in data:
        yield Observe(x, d)
    return mu, sigma

In [41]:
# Sample single point from a distribution.
def distribution_sample(dist, rng):
    if isinstance(dist, Uniform):      return rng.uniform() * (dist.max - dist.min) + dist.min
    elif isinstance(dist, Normal):     return rng.normal() * dist.sigma + dist.mu
    elif isinstance(dist, HalfNormal): return np.abs(rng.normal() * dist.sigma)
    else: assert False

# Calculate log PDF of a distribution at a given point.
def distribution_logp(d, value):
    if isinstance(d, Uniform):
        if d.min <= value <= d.max: return -np.log(d.max - d.min)
        else: return -np.inf
    elif isinstance(d, Normal):
        return -0.5 * np.log(2 * np.pi * d.sigma**2) - (value - d.mu)**2 / (2 * d.sigma ** 2)
    elif isinstance(d, HalfNormal):
        if value >= 0:
            return -0.5 * np.log(np.pi / 2 * d.sigma**2) - value**2 / (2 * d.sigma ** 2)
        else:
            return -np.inf
    else: assert False

In [42]:
# Run a probabilistic program.
def prob_run(program, params=None, rng=None):
    rng = rng if rng is not None else np.random.RandomState(42)
    
    params = params if params is not None else []
    log_p = 0
    index = 0
    
    # The main loop.
    result = None
    action = None
    while True:
        try:
            if action is None:
                action = program.__next__()
            
            action, index, lp = run_action(program, action, params, index, rng)
            log_p += lp
            
        except StopIteration as ex:
            result = ex.value
            break
                    
    return result, params, log_p

In [43]:
def run_action(program, action, params, index, rng):
    log_p = 0
    if isinstance(action, Sample):
        if index < len(params):
            # If we are sampling and we already have a parameter value 
            # for the current trace point.
            r = params[index]
            lp = distribution_logp(action.distribution, r)
            action = program.send(r)
            # log P(θ)
            log_p += lp

            index += 1
        else:
            # Sample a new parameter value.
            r = distribution_sample(action.distribution, rng)
            lp = distribution_logp(action.distribution, r)
            action = program.send(r)
            # log P(θ)
            log_p += lp

            params.append(r)
            index += 1
    elif isinstance(action, Observe):
        # Observing log P(x | θ)
        log_p += distribution_logp(action.distribution, action.value)
        action = None
    else:
        assert False
    return action, index, log_p

In [59]:
def sample_normal(n, obs):
    RNG = np.random.RandomState(42)
    
    # Simulate N(x | mu = 2, sigma = 1)
    observed = RNG.normal(size=obs) * 1 + 2
    
    # Since generators are side-effecting, 
    # we put it in a lambda expression.
    prog = lambda: program(observed)
    
    points = []
    results = []
    r0, x0, p0 = prob_run(prog(), rng=RNG)
    
    for i in range(n):
        points.append(x0)
        results.append(r0)
        
        while True:
            x1 = [x + RNG.normal() * 0.5 for x in x0]
            r1, _, p1 = prob_run(prog(), x1, rng=RNG)
            
            r = p1 - p0
            if r > 0 or np.log(RNG.uniform()) < r:
                x0 = x1
                r0 = r1
                p0 = p1
                break

    results = np.array(results)
    #print(results[:10])
                
    plt.figure(figsize=(4,4))
    plt.xlim([0, 3])
    plt.hist(results[:, 0], bins=100)
    plt.show()
    
    plt.figure(figsize=(4,4))
    plt.xlim([0, 3])
    plt.hist(results[:, 1], bins=100)
    plt.show()

interact(sample_normal, n=(0, 5000, 500),obs=(0,60,1));

interactive(children=(IntSlider(value=2500, description='n', max=5000, step=500), IntSlider(value=30, descript…